# 비동기 방식으로 프로그래밍하려면?
##  asyncio
### async/await 구문을 사용하여 동시성 코드를 작성해 주는 모듈로, 단일 스레드 작업을 병렬로 처리 할 수있다.

In [1]:
import time


def sleep():
    time.sleep(1)


def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end-start}')
    return total


def main():
    start = time.time()

    result1 = sum("A", [1, 2])
    result2 = sum("B", [1, 2, 3])

    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')


if __name__ == "__main__":
    main()

작업중=A, number=1, total=1
작업중=A, number=2, total=3
작업명=A, 걸린시간=2.010179281234741
작업중=B, number=1, total=1
작업중=B, number=2, total=3
작업중=B, number=3, total=6
작업명=B, 걸린시간=3.0205140113830566
총합=9, 총시간=5.030693292617798


In [2]:
# 위 코드는 순차적으로 sum함수를 두번 호출하여 총 5초가 걸렸다
# 위 방식을 비동기 방식으로 바꾸려면? 
# 해당 코드는 .py 파일로 실행해 주세요.
import asyncio
import time


#함수를 비동기로 호출하려면 이렇게 def 앞에 async라는 키워드를 넣으면 된다.
#이때 async를 적용한 비동기 함수를 코루틴이라 부른다.
async def sleep():
    
    #눈여겨봐야 할 점은 sleep() 함수에서 time.sleep(1) 대신 asyncio.sleep(1)를 사용한 부분이다. 
    #코루틴이 아닌 time.sleep(1)을 사용한다면 await가 적용되지 않아 실행 시간을 줄일 수 없다.
    await asyncio.sleep(1)


async def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        
        # 코루틴 안에서 다른 코루틴을 호출할 때는 await sleep()과 같이 await를 함수명 앞에 붙여 호출
        await sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end-start}')
    return total


async def main():
    start = time.time()

    
    #asyncio.create_task()는 수행할 코루틴 작업(태스크)을 생성한다. 여기서는 작업을 생성할 뿐이지 실제로 코루틴이 수행되는 것은 아니다. 
    #실제 코루틴 실행은 await 태스크가 담당한다. 그리고 실행 태스크의 결괏값은 태스크.result()로 얻을 수 있다.
    task1 = asyncio.create_task(sum("A", [1, 2]))
    task2 = asyncio.create_task(sum("B", [1, 2, 3]))

    
    #코루틴 수행 중 await 코루틴을 만나면 await로 호출한 코루틴이 종료될 때까지 기다리지 않고 
    #제어권을 메인 스레드나 다른 코루틴으로 넘긴다.
    #이러한 방식을 넌블록킹(non-blocking)이라 한다. 그리고 호출한 코루틴이 종료되면 이벤트에 의해 다시 그 이후 작업이 수행된다.
    await task1
    await task2

    result1 = task1.result()
    result2 = task2.result()

    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')


if __name__ == "__main__":
    
    #asyncio.run(main())은 런 루프를 생성하여 main() 코루틴을 실행한다. 코루틴을 실행하려면 런 루프가 반드시 필요하다. 
    #코루틴이 모두 비동기적으로 실행되기 때문에 그 시작과 종료를 감지할 수 있는 이벤트 루프가 반드시 필요하기 때문이다.
    asyncio.run(main())
    
    
    
# 실행 결과 

# 작업중=A, number=1, total=1
# 작업중=B, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.031710386276245
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.0728421211242676
# 총합=9, 총시간=3.0738425254821777 

RuntimeError: asyncio.run() cannot be called from a running event loop

# 서버와 통신하는 게임을 만드려면?
## socket
### TCP 서버/클라이언트 프로그램을 작성할 때 사용하는 모듈

1. 서버에서 1~9 사이의 숫자(정답)를 무작위로 생성하고 클라이언트의 접속을 기다린다.
2. 클라이언트는 서버에 접속하여 1~9 사이의 값을 입력하여 게임을 시작한다.
3. 서버는 클라이언트가 입력한 숫자가 정답보다 높을 때는 "너무 높아요"라고, 낮을 때는 "너무 낮아요"라고 응답한다.
4. 클라이언트가 0을 입력하면 "종료"라고 응답하고 서버를 종료한다.
5. 클라이언트가 정답을 입력하면 "정답"이라고 응답하고 서버를 종료한다.

In [5]:
#숫자게임 서버
import socket
import random

HOST = '172.30.1.65'
PORT = 50007

    # 소캣 객체 s 생성 socket.socket(IPv4 인터넷 프로토콜, 소캣이 문자열 등을 주고받는 스트림 방식의 유형)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    #소켓 서버가 HOST라는 IP주소의 PORT 번호에 해당하는 포트로 연결되도록 설정한다는 뜻
    # HOST에는 빈 문자열을 지정했으므로 외부 접속을 허용
    # 만약 HOST에 빈 문자열 대신 'localhost'로 설정한다면 로컬 접속만 허용
    s.bind((HOST, PORT))
    
    
    # 서버 소켓이 클라이언트와의 연결을 시작할 수 있도록 바인딩된 포트를 연다.
    s.listen()
    print('서버가 시작되었습니다.')
    
    #conn, addr = s.accept()는 클라이언트가 접속하면 연결을 수락하고 conn, addr을 반환한다. 
    #이때 conn은 서버와 클라이언트가 연결된 소켓을 의미하고 addr은 클라이언트의 접속 IP를 의미
    conn, addr = s.accept()
    
    
    with conn:
        answer = random.randint(1, 9)
        print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
        while True:
            
            # 클라이언트가 전송한 값을 수신하려면 conn.recv() 함수를 사용하고 클라이언트에 값을 송신하려면 conn.sendall() 함수를 사용한다. 
            #이때 주고받는 메시지는 바이트 문자열이어야 하므로 보낼 때는 UTF-8로 인코딩하고 받을 때에도 UTF-8로 디코딩해야 한다.
            #conn.recv(1024)에서 1024는 한 번에 수신받을 데이터의 최대 바이트 수를 의미하며 보통 1킬로바이트를 의미하는 1,024바이트를 사용한다. 
            #만약 클라이언트로부터 1,024바이트 이상의 데이터를 수신해야 한다면 더 큰 숫자를 사용하거나 루프를 사용하여 
            #conn.recv() 함수를 여러 번 수행해야 한
            data = conn.recv(1024).decode('utf-8')
            print(f'데이터:{data}')

            try:
                n = int(data)
            except ValueError:
                conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
                continue
            
            
            # 클라이언트가 정답 또는 0을 입력할 때까지 클라이언트와 데이터를 주고받는다.
            if n == 0:
                conn.sendall(f"종료".encode('utf-8'))
                break
            if n > answer:
                conn.sendall("너무 높아요".encode('utf-8'))
            elif n < answer:
                conn.sendall("너무 낮아요".encode('utf-8'))
            else:
                conn.sendall("정답".encode('utf-8'))
                break

서버가 시작되었습니다.
클라이언트가 접속했습니다:('172.30.1.65', 52054), 정답은 2 입니다.
데이터:GET / HTTP/1.1
Host: 172.30.1.65:50007
Connection: keep-alive
Cache-Control: max-age=0
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7


데이터:


ConnectionAbortedError: [WinError 10053] 현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다

In [ ]:
# 숫자게임 클라이언트 

import socket

HOST = 'localhost'
PORT = 50007

# 서버 소켓에서는 연결된 클라이언트를 의미하는 conn 객체를 통해 데이터를 주고받았다면
#소켓 클라이언트에서는 서버와 연결을 의미하는 소켓 객체인 s 객체를 통해 데이터를 주고받는다
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    while True:
        n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
        if not n.strip():
            print("입력값이 잘못되었습니다.")
            continue
        s.sendall(n.encode('utf-8'))
        data = s.recv(1024).decode('utf-8')
        print(f'서버응답:{data}')
        if data == "정답" or data == "종료":
            break

# SSL로 서버와 통신하려면?
## ssl
### socat 모듈로 작성한 서버/클라이언트에 공개키 암호화 방식을 적용할 때 사용하는 모듈

https://wikidocs.net/125373

이번 예제는 리눅스 환경에서의 예제밖에 없어서 Pass

# 여러 명이 동시에 접속하려면?
## select
### 소켓 프로그래밍에서 I/O 멀티플렉싱을 가능하게 하는 모듈

In [ ]:
# 클라이언트 요청을 동시에 처리하여 한꺼번에 여러 명이 동시에 플레이할 수 있도록 
#기존 소켓 서버 프로그램을 수정하려면 어떻게 해야 할까?

import socket
import select
import random

HOST = ''
PORT = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print('서버가 시작되었습니다.')

    #최초 readsocks에는 서버 소켓만 있기 때문에 select는 이 서버 소켓에 클라이언트가 접속하는지 감시
    readsocks = [s]
    
    
    # 클라이언트마다 답을 다르게 하기위해 {클라이언트 소켓 : 답} 딕셔너리로 구성
    answers = {}

    while True:
        
        # 가장 핵심적인 부분 readsocks에 포함된 소켓에서 이벤트가 발생하는지 감시하는 역할
        # 이벤트가 발생하면 이후 이 문장을 실행
        readables, writeables, excpetions = select.select(readsocks, [], [])
        #readables는 수신한 데이터를 가진 소켓을 의미하고 writeables는 블로킹되지 않고 데이터를 전송할 수 있는 
        #소켓을, exceptions는 예외 상황이 발생한 소켓을 의미
        #readables, writeables, exceptions는 모두 여러 개의 소켓으로 리스트를 구성
        
        for sock in readables:
            
            #신규 접속이 아니라 이미 접속한 클라이언트에서 
            #데이터 요청 등의 이벤트가 발생하면 if sock == s:가 거짓(False)이 되어 해당 클라이언트와 숫자 게임을 진행
            if sock == s:  # 신규 클라이언트 접속
                newsock, addr = s.accept()
                answer = random.randint(1, 9)
                print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
                
                #신규 클라이언트가 서버 소켓에 접속하면 readsocks에 신규 클라이언트의 소켓을 추가 
                #readsocks에는 서버 소켓 외에 클라이언트 소켓도 포함
                readsocks.append(newsock)
                answers[newsock] = answer  # 클라이언트 별 정답 생성
            else:  # 이미 접속한 클라이언트의 요청 (게임진행을 위한 요청)
                conn = sock
                data = conn.recv(1024).decode('utf-8')
                print(f'데이터:{data}')

                try:
                    n = int(data)
                except ValueError:
                    conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
                    continue

                answer = answers.get(conn)
                if n == 0:
                    conn.sendall(f"종료".encode('utf-8'))
                    conn.close()
                    readsocks.remove(conn)  # 클라이언트 접속 해제시 readsocks에서 제거
                if n > answer:
                    conn.sendall("너무 높아요".encode('utf-8'))
                elif n < answer:
                    conn.sendall("너무 낮아요".encode('utf-8'))
                else:
                    conn.sendall("정답".encode('utf-8'))
                    conn.close()
                    readsocks.remove(conn)  # 클라이언트 접속 해제시 readsocks에서 제거

서버가 시작되었습니다.
클라이언트가 접속했습니다:('127.0.0.1', 52112), 정답은 5 입니다.
데이터:5


# 멀티 플레이 게임 서버를 업그레이드 하려면?
## selector
### select를 확장하여 고수준 I/O 멀티 플렉싱을 가능하도록 한 모듈, selet 대신 사용하도록 권장하는 모듈

In [ ]:
# 숫자 게임 서버 selectors 모듈을 사용
import socket
import selectors
import random

HOST = ''
PORT = 50007

sel = selectors.DefaultSelector()  # 최적의 Selector를 생성한다.
answers = {}


#selectors 모듈은 DefaultSelector로 생성한 객체에 이벤트(실행할 함수)를 등록해야 하는 구조


#클라이언트의 접속을 처리하는 함수 accept_client()
def accept_client(sock):
    """ 서버 소켓에 클라이언트가 접속하면 호출된다. """
    conn, addr = sock.accept()
    answer = random.randint(1, 9)
    answers[conn] = answer
    
    #el.register(conn, selectors.EVENT_READ, game_client)로 클라이언트 소켓에 데이터를 수신하면 game_client() 함수가 실행되도록 설정
    sel.register(conn, selectors.EVENT_READ, game_client)  # 클라이언트 소켓을 등록한다.
    print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')

    
    
#클라이언트와 게임을 진행하는 함수 game_client()
def game_client(conn):
    """ 클라이언트 소켓에 데이터가 수신되면 호출된다. """
    data = conn.recv(1024).decode('utf-8')
    print(f'데이터:{data}')
    try:
        n = int(data)
        answer = answers.get(conn)
        if n == 0:
            conn.sendall(f"종료".encode('utf-8'))
            sel.unregister(conn)
            conn.close()
        elif n > answer:
            conn.sendall("너무 높아요".encode('utf-8'))
        elif n < answer:
            conn.sendall("너무 낮아요".encode('utf-8'))
        else:
            conn.sendall("정답".encode('utf-8'))
            sel.unregister(conn)
            conn.close()
    except ValueError:
        conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print('서버가 시작되었습니다.')
    sel.register(s, selectors.EVENT_READ, accept_client)  # 서버 소켓을 등록한다.

    while True:
        events = sel.select()  # 클라이언트의 접속 또는 접속된 클라이언트의 데이터 요청을 감시
        for key, mask in events:
            callback = key.data  # 실행할 함수
            callback(key.fileobj)  # 이벤트가 발생한 소켓을 인수로 실행할 함수를 실행한다.

# 사용자가 보낸 신호를 처리하려면?
## signal
### 특정 신호를 수신했을 때 사용자가 정의한 함수를 호출하도록 한 모듈이다.

In [0]:
#다음은 10초에 한번씩 대기중... 을 출력하는 프로그램 
import time

while True:
    print('대기중...')
    time.sleep(10)

대기중...
대기중...
대기중...
대기중...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-9804a492b3d3>", line 6, in <module>
    time.sleep(10)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  

TypeError: object of type 'NoneType' has no len()

In [ ]:
#Control + C 나 인터럽트시 다음과같이 프로그램이 중단
# 사용자가 실수로 또는 고의로 Ctrl+C를 입력하더라도 프로그램이 중단되지 않도록 하려면 어떻게 해야 할까?

import time
import signal


def handler(signum, frame):
    print("Ctrl+C 신호를 수신했습니다.")


signal.signal(signal.SIGINT, handler)

while True:
    print('대기중...')
    time.sleep(10)

대기중...
ERROR! Session/line number was not unique in database. History logging moved to new session 672
Ctrl+C 신호를 수신했습니다.
대기중...
Ctrl+C 신호를 수신했습니다.
대기중...
